In [1]:
import pandas as pd
import sys
import math
from collections import defaultdict

infile='/Users/irffanalahi/Research/Research_code/gitignorefolder/target_panel/intro_exon_bp/intron_genecode_v36_modified.txt_canonical.txt_proteinCodingGene.txt'
outfile=infile+"_bpscaled"

origindf=pd.read_csv(infile,sep="\t")
indf=origindf.copy()
indf.head()

,#hg38.knownCanonical.chrom_x,hg38.knownCanonical.chromStart_x,hg38.knownCanonical.chromEnd_x,hg38.knownCanonical.transcript,transcriptno,direction,#hg38.knownCanonical.chrom_y,hg38.knownCanonical.chromStart_y,hg38.knownCanonical.chromEnd_y,hg38.kgXref.geneSymbol
0,chr1,925800,925921,ENST00000342066.8,0,+,chr1,925730,944574,SAMD11
1,chr1,926013,930154,ENST00000342066.8,1,+,chr1,925730,944574,SAMD11
2,chr1,930336,931038,ENST00000342066.8,2,+,chr1,925730,944574,SAMD11
3,chr1,931089,935771,ENST00000342066.8,3,+,chr1,925730,944574,SAMD11
4,chr1,935896,939039,ENST00000342066.8,4,+,chr1,925730,944574,SAMD11


In [2]:
indf.drop_duplicates(subset=['#hg38.knownCanonical.chrom_x','hg38.knownCanonical.chromStart_x','hg38.knownCanonical.chromEnd_x','hg38.knownCanonical.transcript','transcriptno'],inplace=True)
indf[indf.duplicated(subset=['hg38.knownCanonical.transcript','transcriptno'])]

,#hg38.knownCanonical.chrom_x,hg38.knownCanonical.chromStart_x,hg38.knownCanonical.chromEnd_x,hg38.knownCanonical.transcript,transcriptno,direction,#hg38.knownCanonical.chrom_y,hg38.knownCanonical.chromStart_y,hg38.knownCanonical.chromEnd_y,hg38.kgXref.geneSymbol


In [3]:
indf['feature_size']=indf['hg38.knownCanonical.chromEnd_x']-indf['hg38.knownCanonical.chromStart_x']
indf['totalbp_forcorrespondingTranscript']=-1
indf['bp_percentage']=-1
indf['bp_inscaled_block']=-1

indf['startingtranNumber']=-1

indf.head()

,#hg38.knownCanonical.chrom_x,hg38.knownCanonical.chromStart_x,hg38.knownCanonical.chromEnd_x,hg38.knownCanonical.transcript,transcriptno,direction,#hg38.knownCanonical.chrom_y,hg38.knownCanonical.chromStart_y,hg38.knownCanonical.chromEnd_y,hg38.kgXref.geneSymbol,feature_size,totalbp_forcorrespondingTranscript,bp_percentage,bp_inscaled_block,startingtranNumber
0,chr1,925800,925921,ENST00000342066.8,0,+,chr1,925730,944574,SAMD11,121,-1,-1,-1,-1
1,chr1,926013,930154,ENST00000342066.8,1,+,chr1,925730,944574,SAMD11,4141,-1,-1,-1,-1
2,chr1,930336,931038,ENST00000342066.8,2,+,chr1,925730,944574,SAMD11,702,-1,-1,-1,-1
3,chr1,931089,935771,ENST00000342066.8,3,+,chr1,925730,944574,SAMD11,4682,-1,-1,-1,-1
4,chr1,935896,939039,ENST00000342066.8,4,+,chr1,925730,944574,SAMD11,3143,-1,-1,-1,-1


In [4]:
grtranscript=indf.groupby(['hg38.knownCanonical.transcript'])

for name, group in grtranscript:
    totalbp=group['feature_size'].sum()
    indf.loc[group.index,'totalbp_forcorrespondingTranscript']=totalbp
indf.head()

,#hg38.knownCanonical.chrom_x,hg38.knownCanonical.chromStart_x,hg38.knownCanonical.chromEnd_x,hg38.knownCanonical.transcript,transcriptno,direction,#hg38.knownCanonical.chrom_y,hg38.knownCanonical.chromStart_y,hg38.knownCanonical.chromEnd_y,hg38.kgXref.geneSymbol,feature_size,totalbp_forcorrespondingTranscript,bp_percentage,bp_inscaled_block,startingtranNumber
0,chr1,925800,925921,ENST00000342066.8,0,+,chr1,925730,944574,SAMD11,121,16287,-1,-1,-1
1,chr1,926013,930154,ENST00000342066.8,1,+,chr1,925730,944574,SAMD11,4141,16287,-1,-1,-1
2,chr1,930336,931038,ENST00000342066.8,2,+,chr1,925730,944574,SAMD11,702,16287,-1,-1,-1
3,chr1,931089,935771,ENST00000342066.8,3,+,chr1,925730,944574,SAMD11,4682,16287,-1,-1,-1
4,chr1,935896,939039,ENST00000342066.8,4,+,chr1,925730,944574,SAMD11,3143,16287,-1,-1,-1


In [5]:
indf['bp_percentage']=indf['feature_size']/indf['totalbp_forcorrespondingTranscript']*100
indf['bp_percentage']=indf['bp_percentage'].round()


indf.to_csv(outfile+"_likelog_first.txt",sep="\t",index=False)
indf.head()

,#hg38.knownCanonical.chrom_x,hg38.knownCanonical.chromStart_x,hg38.knownCanonical.chromEnd_x,hg38.knownCanonical.transcript,transcriptno,direction,#hg38.knownCanonical.chrom_y,hg38.knownCanonical.chromStart_y,hg38.knownCanonical.chromEnd_y,hg38.kgXref.geneSymbol,feature_size,totalbp_forcorrespondingTranscript,bp_percentage,bp_inscaled_block,startingtranNumber
0,chr1,925800,925921,ENST00000342066.8,0,+,chr1,925730,944574,SAMD11,121,16287,1.0,-1,-1
1,chr1,926013,930154,ENST00000342066.8,1,+,chr1,925730,944574,SAMD11,4141,16287,25.0,-1,-1
2,chr1,930336,931038,ENST00000342066.8,2,+,chr1,925730,944574,SAMD11,702,16287,4.0,-1,-1
3,chr1,931089,935771,ENST00000342066.8,3,+,chr1,925730,944574,SAMD11,4682,16287,29.0,-1,-1
4,chr1,935896,939039,ENST00000342066.8,4,+,chr1,925730,944574,SAMD11,3143,16287,19.0,-1,-1


In [6]:

#indf.rename(columns={'bp_percentage':'orig_bp_percentage'},inplace=True)

#indf['bp_percentage']=indf['orig_bp_percentage'].copy()
indf=indf[indf['bp_percentage']>0].copy()
indf.reset_index(drop=True,inplace=True)

indf.head()

,#hg38.knownCanonical.chrom_x,hg38.knownCanonical.chromStart_x,hg38.knownCanonical.chromEnd_x,hg38.knownCanonical.transcript,transcriptno,direction,#hg38.knownCanonical.chrom_y,hg38.knownCanonical.chromStart_y,hg38.knownCanonical.chromEnd_y,hg38.kgXref.geneSymbol,feature_size,totalbp_forcorrespondingTranscript,bp_percentage,bp_inscaled_block,startingtranNumber
0,chr1,925800,925921,ENST00000342066.8,0,+,chr1,925730,944574,SAMD11,121,16287,1.0,-1,-1
1,chr1,926013,930154,ENST00000342066.8,1,+,chr1,925730,944574,SAMD11,4141,16287,25.0,-1,-1
2,chr1,930336,931038,ENST00000342066.8,2,+,chr1,925730,944574,SAMD11,702,16287,4.0,-1,-1
3,chr1,931089,935771,ENST00000342066.8,3,+,chr1,925730,944574,SAMD11,4682,16287,29.0,-1,-1
4,chr1,935896,939039,ENST00000342066.8,4,+,chr1,925730,944574,SAMD11,3143,16287,19.0,-1,-1


In [7]:
def populate_rows(arow):
    every_MAXblock_size=math.ceil(arow['bp_inscaled_block'])
    every_MINblock_size=math.floor(arow['bp_inscaled_block'])
    totalblocks=arow['bp_percentage']
    
    adjustedMAXtotalbp=every_MAXblock_size*totalblocks
    
    adjustedMINtotalbp=every_MINblock_size*totalblocks
    
    bpadded=adjustedMAXtotalbp-arow['feature_size']
    bpdiscarded=arow['feature_size']-adjustedMINtotalbp
    
 
    
    MAXblock_start=arow['hg38.knownCanonical.chromStart_x']-math.ceil(bpadded/2)
    if MAXblock_start<0:
        MAXblock_start=0
    
    MAXblockend=arow['hg38.knownCanonical.chromEnd_x']+math.floor(bpadded/2)
    
    
    MINblockstart=arow['hg38.knownCanonical.chromStart_x']+math.floor(bpdiscarded/2)
    MINblockend=arow['hg38.knownCanonical.chromEnd_x']-math.ceil(bpdiscarded/2)
    
    

    '''
    if MINblockstart>=MINblockend:
        print(arow)
        print("whatttttttt?")
        sys.exit(1)
    '''
    if (MAXblockend-MAXblock_start)/every_MAXblock_size!=totalblocks:
        print(arow)
        print("whatttttttt MAXblocks?")
        sys.exit(1)
    
    '''
    if (MINblockend-MINblockstart)/every_MINblock_size!=totalblocks:
        print(arow)
        print("whatttttttt MINblocks?")
        sys.exit(1)
    '''
    
    
    #print(bpadded)
    #print(bpdiscarded)
    
    #print(MAXblock_start,MAXblockend,(MAXblockend-MAXblock_start)/every_MAXblock_size)
    
    MAXblockdf=justpopulation(arow['#hg38.knownCanonical.chrom_x'],MAXblock_start,MAXblockend,every_MAXblock_size,totalblocks,arow['hg38.knownCanonical.transcript'],arow['startingtranNumber'],arow['direction'])
   

    #print(MINblockstart,MINblockend,(MINblockend-MINblockstart)/every_MINblock_size)
    
    #MINblockdf=justpopulation(arow['#hg38.knownCanonical.chrom_x'],MINblockstart,MINblockend,every_MINblock_size,totalblocks,arow['hg38.knownCanonical.transcript'],arow['startingtranNumber'])
    #print(MINblockdf)
    
    return MAXblockdf #,MINblockdf

    
def justpopulation(blockchr,blockstart,blockend,blocksize,howmany,transcript,startblocknumber,direction):
    outdict = defaultdict(list)
    currentstart=blockstart
    blockcoungter=startblocknumber
    
   
    while currentstart<blockend:
        currentend=currentstart+blocksize
        outdict['#hg38.knownCanonical.chrom_x'].append(blockchr)
        outdict['hg38.knownCanonical.chromStart_x'].append(currentstart)
        outdict['hg38.knownCanonical.chromEnd_x'].append(currentend)
        outdict['hg38.knownCanonical.transcript'].append(transcript)
        outdict['transcriptno'].append(blockcoungter)
        outdict['direction'].append(direction)
        
        
        blockcoungter=blockcoungter+1
        currentstart=currentend
        
    '''    
    if currentstart!=blockend:
        print(currentstart,blockcoungter)
        print(blockchr,blockstart,blockend,blocksize,howmany)
        print("justpopulation, currentstart!=blockend")
        sys.exit(1)
    '''
    
    '''
    if blockcoungter!=(howmany+startblocknumber):
        print(blockchr,blockstart,blockend,blocksize,howmany)
        print("justpopulation, blockcoungter!=howmany")
        sys.exit(1)
    '''
    
    outdf=pd.DataFrame.from_dict(outdict)
    
    return outdf
        
    

    
    

In [8]:
for index, row in indf.iterrows():
    if row['transcriptno']==0:
        indf.loc[index,'startingtranNumber']=0
    
    
    elif indf.loc[index-1,'hg38.knownCanonical.transcript']!=row['hg38.knownCanonical.transcript']:
        indf.loc[index,'startingtranNumber']=0
    
    else:

     
        
        if indf.loc[index-1,'hg38.knownCanonical.chromStart_x']>=row['hg38.knownCanonical.chromStart_x']:
            print("not sorted?")
            sys.exit(1)
        
        
        
        if indf.loc[index-1,'transcriptno']>=row['transcriptno']:
            print("my transcriptno assumption wrong?")
            sys.exit(1)
        
        
        prevstartingtranNumber=indf.loc[index-1,'startingtranNumber']
        
        if prevstartingtranNumber==-1:
            print("prevstartingtranNumber not set")
            sys.exit(1)
        
        prevtotalblock=indf.loc[index-1,'bp_percentage']
        
        nextstartnumber=prevstartingtranNumber+prevtotalblock
        indf.loc[index,'startingtranNumber']=nextstartnumber

indf['bp_inscaled_block']=indf['feature_size']/indf['bp_percentage']
indf.to_csv(outfile+"_likelog_second.txt",sep="\t",index=False)
indf.head()

,#hg38.knownCanonical.chrom_x,hg38.knownCanonical.chromStart_x,hg38.knownCanonical.chromEnd_x,hg38.knownCanonical.transcript,transcriptno,direction,#hg38.knownCanonical.chrom_y,hg38.knownCanonical.chromStart_y,hg38.knownCanonical.chromEnd_y,hg38.kgXref.geneSymbol,feature_size,totalbp_forcorrespondingTranscript,bp_percentage,bp_inscaled_block,startingtranNumber
0,chr1,925800,925921,ENST00000342066.8,0,+,chr1,925730,944574,SAMD11,121,16287,1.0,121.000000,0.0
1,chr1,926013,930154,ENST00000342066.8,1,+,chr1,925730,944574,SAMD11,4141,16287,25.0,165.640000,1.0
2,chr1,930336,931038,ENST00000342066.8,2,+,chr1,925730,944574,SAMD11,702,16287,4.0,175.500000,26.0
3,chr1,931089,935771,ENST00000342066.8,3,+,chr1,925730,944574,SAMD11,4682,16287,29.0,161.448276,30.0
4,chr1,935896,939039,ENST00000342066.8,4,+,chr1,925730,944574,SAMD11,3143,16287,19.0,165.421053,59.0


In [9]:
MAXblocklist=[]
#MINblocklist=[]
for index, row in indf.iterrows():
     
        
    
    tmpMAXblock=populate_rows(row)
    
    MAXblocklist.append(tmpMAXblock)
  
    #MINblocklist.append(tmpMINblock)

    
MAXmasterdf=pd.concat(MAXblocklist)
#MINmasterdf=pd.concat(MINblocklist)




In [10]:
MAXmasterdf.to_csv(outfile+"_MAX.txt",sep="\t",index=False)
#MINmasterdf.to_csv(outfile+"_MIN.txt",sep="\t",index=False)